# Construya un conjunto de datos de imágenes en TensorFlow.

Para este ejemplo, necesita crear su propio conjunto de imágenes (JPEG).
Mostraremos 2 maneras diferentes de construir ese conjunto de datos:

- Desde una carpeta raíz, que tendrá una subcarpeta que contendrá imágenes para cada clase

```
    ROOT_FOLDER
       |-------- SUBFOLDER (CLASS 0)
       |             |
       |             | ----- image1.jpg
       |             | ----- image2.jpg
       |             | ----- etc...
       |             
       |-------- SUBFOLDER (CLASS 1)
       |             |
       |             | ----- image1.jpg
       |             | ----- image2.jpg
       |             | ----- etc...

```

- Desde un archivo de texto plano, que mostrará todas las imágenes con su ID de clase:

```
    /path/to/image/1.jpg CLASS_ID
    /path/to/image/2.jpg CLASS_ID
    /path/to/image/3.jpg CLASS_ID
    /path/to/image/4.jpg CLASS_ID
    etc...
```

A continuación, hay algunos parámetros que necesita cambiar (Marcado 'CHANGE HERE'), 
como la ruta del conjunto de datos.

In [ ]:
from __future__ import print_function

import tensorflow as tf
import os

# Parámetros del conjunto de datos - CAMBIAR AQUÍ
MODE = 'folder' # o 'file', si elige un archivo de texto plano (ver arriba).
DATASET_PATH = '/path/to/dataset/' # el archivo del conjunto de datos o la ruta de la carpeta raíz.

# Parámetros de imagen
N_CLASSES = 2 # CAMBIAR AQUÍ, número total de clases
IMG_HEIGHT = 64 # CAMBIAR AQUÍ, la altura de la imagen a redimensionadar
IMG_WIDTH = 64 # CAMBIAR AQUÍ, el ancho de la imagen a redimensionar
CHANNELS = 3 # Los 3 canales de color, cambie a 1 si la escala de grises

In [ ]:
# Leyendo el conjunto de datos
# 2 modos: 'file' o 'folder'.
def read_images(dataset_path, mode, batch_size):
    imagepaths, labels = list(), list()
    if mode == 'file':
        # Leer el archivo del conjunto de datos
        data = open(dataset_path, 'r').read().splitlines()
        for d in data:
            imagepaths.append(d.split(' ')[0])
            labels.append(int(d.split(' ')[1]))
    elif mode == 'folder':
        # Un ID será afectado a cada subcarpeta por orden alfabético
        label = 0
        # Liste el directorio
        try:  # Python 2
            classes = sorted(os.walk(dataset_path).next()[1])
        except Exception:  # Python 3
            classes = sorted(os.walk(dataset_path).__next__()[1])
        # Liste cada subdirectorio (las clases)
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            try:  # Python 2
                walk = os.walk(c_dir).next()
            except Exception:  # Python 3
                walk = os.walk(c_dir).__next__()
            # Añade cada imagen al kit de entrenamiento
            for sample in walk[2]:
                # Sólo mantiene imágenes jpeg
                if sample.endswith('.jpg') or sample.endswith('.jpeg'):
                    imagepaths.append(os.path.join(c_dir, sample))
                    labels.append(label)
            label += 1
    else:
        raise Exception("Unknown mode.")

    # Convertir a Tensor
    imagepaths = tf.convert_to_tensor(imagepaths, dtype=tf.string)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)
    # Construir una cola TF, mezclar datos
    image, label = tf.train.slice_input_producer([imagepaths, labels],
                                                 shuffle=True)

    # Leer imágenes del disco
    image = tf.read_file(image)
    image = tf.image.decode_jpeg(image, channels=CHANNELS)

    # Redimensionar las imágenes a un tamaño común
    image = tf.image.resize_images(image, [IMG_HEIGHT, IMG_WIDTH])

    # Normalizar
    image = image * 1.0/127.5 - 1.0

    # Crear Lotes
    X, Y = tf.train.batch([image, label], batch_size=batch_size,
                          capacity=batch_size * 8,
                          num_threads=4)

    return X, Y

In [ ]:
# -----------------------------------------------
# ESTA ES UNA CNN CLÁSICA (ver ejemplos, sección 3)
# -----------------------------------------------
# Tenga en cuenta que algunos elementos han cambiado (uso de colas).

# Parametros
learning_rate = 0.001
num_steps = 10000
batch_size = 128
display_step = 100

# Network Parametros
dropout = 0.75 # Abandono, probabilidad de mantener las unidades

# Construir la entrada de datos
X, Y = read_images(DATASET_PATH, MODE, batch_size)

In [ ]:
# Crear modelo
def conv_net(x, n_classes, dropout, reuse, is_training):
    # Definir un ámbito de reutilización de las variables
    with tf.variable_scope('ConvNet', reuse=reuse):

        # Capa de convolución con 32 filtros y un tamaño de kernel de 5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # Agrupación máxima (muestreo descendente) con zancadas de 2 y tamaño de kernel de 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Capa de convolución con 32 filtros y un tamaño de kernel de 5
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Agrupación máxima (muestreo descendente) con zancadas de 2 y tamaño de kernel de 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # Aplanar los datos a un vector 1-D para la capa completamente conectada.
        fc1 = tf.contrib.layers.flatten(conv2)

        # Capa completamente conectada (en la carpeta contrib por ahora)
        fc1 = tf.layers.dense(fc1, 1024)
        # Aplicar Abandono (si is_training es Falso, no se aplica el abandono)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Capa de salida, predicción de clases
        out = tf.layers.dense(fc1, n_classes)
        # Porque'softmax_cross_entropy_with_logits' ya aplica softmax,
        # sólo aplicamos softmax a la red de pruebas
        out = tf.nn.softmax(out) if not is_training else out

    return out

In [ ]:
# Debido a que los Abandonados tienen un comportamiento diferente en el entrenamiento y en el tiempo de predicción, nosotros
# necesitan crear 2 gráficos de cálculo distintos que compartan los mismos pesos.

# Cree un gráfico para el entrenamiento
logits_train = conv_net(X, N_CLASSES, dropout, reuse=False, is_training=True)
# Cree otro gráfico para probar que reutilice los mismos pesos
logits_test = conv_net(X, N_CLASSES, dropout, reuse=True, is_training=False)

# Definir la pérdida y el optimizador (con los registros del tren, para que la pérdida de datos tenga efecto)
loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=logits_train, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluar el modelo (con registros de pruebas, para deshabilitar la deserción escolar)
correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.cast(Y, tf.int64))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Inicializar las variables (es decir, asignar su valor por defecto)
init = tf.global_variables_initializer()

# Objeto Saver
saver = tf.train.Saver()

# Iniciar entrenamiento
with tf.Session() as sess:

    # Ejecutar el inicializador
    sess.run(init)

    # Iniciar la cola de datos
    tf.train.start_queue_runners()

    # Circulo de entrenamiento
    for step in range(1, num_steps+1):

        if step % display_step == 0:
            # Optimización de la ejecución y cálculo de la pérdida y precisión de los lotes
            _, loss, acc = sess.run([train_op, loss_op, accuracy])
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        else:
            # Ejecute únicamente la optimización op (backprop)
            sess.run(train_op)

    print("Optimization Finished!")

    # Guardar el modelo
    saver.save(sess, 'my_tf_model')